In [1]:
import configparser
import tweepy
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import time

In [2]:
# get tokens
config = configparser.ConfigParser()
config.read('configfile.ini')
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
# authenticate
auth = tweepy.OAuth2AppHandler(api_key, api_key_secret)

In [3]:
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
def get_tweets_dataframe(working_df, tweets, hashtag):
    #working_df=working_df.drop_duplicates()
    index=len(working_df)
    for tweet in tweets: 
        working_df.loc[index,'tweet_id']=tweet.id
        working_df.loc[index,'created_at']=tweet.created_at
        working_df.loc[index,'user']=tweet.user.screen_name
        working_df.loc[index,'full_text']=tweet.full_text
        working_df.loc[index,'favorite_count']=tweet.favorite_count
        working_df.loc[index,'retweet_count']=tweet.retweet_count
        working_df.loc[index,'hashtags']=hashtag
        print(working_df.loc[index,'created_at'])
        #working_df=working_df.drop_duplicates()
        index+=1
        working_df.to_csv(f'tweeterhashtags/{hashtag}.csv', index=False)
        #time.sleep(1)
    working_df=working_df.drop_duplicates()
    working_df.to_csv(f'tweeterhashtags/{hashtag}.csv', index=False)               
    return working_df

In [5]:
hashtagsweek1 = ['#FridayNightatPortlandRow', '#HauntedWatchParty', '#WatchPartyatPortlandRow', '#HauntedbyaType3','#TogetherForLockwoodandCo','#PrimeForLockwoodandCo','#BringBackLockwoodandCo']
hashtagsweek2 = ['#GhostHuntersWatchParty', '#DisneyForLockwoodandCo', '#BBCforLockwoodandCo', '#AppleTVforLockwoodandCo', '#PrimeForLockwoodandCo', '#JustRecklessEnough']
hashtagsweek3 = ['#LockwoodGhostAuditions', '#ParamountForLockwoodandCo', '#ScullandCo','#RapiersReady', '#CaringforCarlyle', '#DEPRACisOnTheWay', '#BunsForBunchurch']
hashtagsweek4 = ['#CompleteFictionAppreciation', '#DisneySaveLockwood', '#ArtistryofLockwoodandCo', '#ParamountSaveLockwood', '#GhostStrike', '#LockwoodParallelFandoms', '#JustRecklessEnough']
hashtagsall = hashtagsweek1+hashtagsweek2+hashtagsweek3+hashtagsweek4
hashtagsall = list(set(hashtagsall))

In [6]:
hashtags=['#DisneySaveLockwood']
for hashtag in hashtags:
    print(hashtag)
    try:
        working_df=pd.read_csv(f'tweeterhashtags/{hashtag}.csv')
        #working_df = pd.read_csv('tweeterhashtags/#SkullandCo.csv')
        print(len(working_df))
    except:
        working_df=pd.DataFrame(columns=['tweet_id','created_at', 'user', 'full_text','favorite_count','retweet_count','hashtags'])
    tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()
    working_df = get_tweets_dataframe(working_df, tweets, hashtag)
    working_df=working_df.drop_duplicates()
    print(len(working_df))
    time.sleep(60)
    working_df['created_at'] = pd.to_datetime(working_df['created_at'], utc=True)
    # Group the DataFrame by day
    grouped_df = working_df.groupby(pd.Grouper(key='created_at', freq='D'))

    # Calculate the count of rows for each day
    count_per_day = grouped_df['hashtags'].count()

    # Display the count per day
    print(count_per_day)


#DisneySaveLockwood
3920
2023-06-08 10:12:07+00:00
2023-06-08 08:59:04+00:00
2023-06-08 08:57:49+00:00
2023-06-08 08:37:56+00:00
2023-06-08 07:59:52+00:00
2023-06-08 07:59:46+00:00
2023-06-08 07:22:47+00:00
2023-06-08 06:45:52+00:00
2023-06-08 06:08:13+00:00
2023-06-08 04:54:10+00:00
2023-06-08 04:50:45+00:00
2023-06-08 04:44:31+00:00
2023-06-08 02:52:47+00:00
2023-06-08 02:50:57+00:00
2023-06-08 02:19:38+00:00
2023-06-08 02:14:19+00:00
2023-06-08 02:14:01+00:00
2023-06-08 02:13:49+00:00
2023-06-08 02:12:45+00:00
2023-06-08 02:12:38+00:00
2023-06-08 02:11:22+00:00
2023-06-08 02:04:40+00:00
2023-06-08 02:04:04+00:00
2023-06-08 02:02:28+00:00
2023-06-08 01:49:57+00:00
2023-06-08 01:22:48+00:00
2023-06-08 01:21:59+00:00
2023-06-08 01:20:42+00:00
2023-06-08 01:17:14+00:00
2023-06-08 01:11:19+00:00
2023-06-08 01:11:12+00:00
2023-06-08 01:07:21+00:00
2023-06-08 01:06:56+00:00
2023-06-08 00:59:08+00:00
2023-06-08 00:52:04+00:00
2023-06-08 00:50:50+00:00
2023-06-08 00:43:45+00:00
2023-06-08 00

In [7]:
working_df=pd.DataFrame()
for hashtag in hashtagsall:
    try:
        working_df=pd.concat([working_df, pd.read_csv(f'tweeterhashtags/{hashtag}.csv')], ignore_index=True)
    except:
        pass
working_df

,tweet_id,created_at,user,full_text,favorite_count,retweet_count,hashtags,tweet_df
0,1.663840e+18,2023-05-31 09:18:34+00:00,Lockwood1494,@Yourmother50 @ParamountUK @paramountplus Alre...,0.0,0.0,#SkullandCo,NaN
1,1.663840e+18,2023-05-31 09:18:16+00:00,lovinpages,"RT @Earthling738: I have to say, this is still...",0.0,2.0,#SkullandCo,NaN
2,1.663840e+18,2023-05-31 09:17:55+00:00,Lockwood1494,"RT @Earthling738: I have to say, this is still...",0.0,2.0,#SkullandCo,NaN
3,1.663840e+18,2023-05-31 09:16:30+00:00,nemesisn15,@alice_mirror_ @ababahalamaha Your edition is ...,1.0,0.0,#SkullandCo,NaN
4,1.663840e+18,2023-05-31 09:15:49+00:00,Earthling738,"I have to say, this is still one of my favorit...",2.0,2.0,#SkullandCo,NaN
...,...,...,...,...,...,...,...,...
118802,1.665265e+18,2023-06-04 07:49:42+00:00,VicthoriaAnne,RT @junebugjan2: I love their dynamic so much\...,0.0,20.0,#DEPRACisOnTheWay,NaN
118803,1.665264e+18,2023-06-04 07:48:55+00:00,VicthoriaAnne,RT @Lulu_Here_: AHHHHHHH keep sharing the peti...,0.0,8.0,#DEPRACisOnTheWay,NaN
118804,1.665264e+18,2023-06-04 07:46:08+00:00,emmyyyg__,"RT @mixleven: i need more trio content, i’m de...",0.0,26.0,#DEPRACisOnTheWay,NaN
118805,1.665263e+18,2023-06-04 07:42:52+00:00,hollowgirlhan,RT @Iff_ustin: I got this!! Flattered cuz i fe...,0.0,1.0,#DEPRACisOnTheWay,NaN


In [8]:
working_df['full_text'] = working_df['full_text'].str.replace(r'"', '')
working_df['full_text']

0         @Yourmother50 @ParamountUK @paramountplus Alre...
1         RT @Earthling738: I have to say, this is still...
2         RT @Earthling738: I have to say, this is still...
3         @alice_mirror_ @ababahalamaha Your edition is ...
4         I have to say, this is still one of my favorit...
                                ...                        
118802    RT @junebugjan2: I love their dynamic so much\...
118803    RT @Lulu_Here_: AHHHHHHH keep sharing the peti...
118804    RT @mixleven: i need more trio content, i’m de...
118805    RT @Iff_ustin: I got this!! Flattered cuz i fe...
118806    RT @RebekahBlazek: Does Lucy have charms on he...
Name: full_text, Length: 118807, dtype: object

In [9]:
# Create a new column 'retweet' with default value False
working_df['retweet'] = False

# Check if 'full_text' starts with 'RT ' and set 'retweet' column accordingly
working_df.loc[working_df['full_text'].str.startswith('RT '), 'retweet'] = True

working_df['retweet'].value_counts()

True     62272
False    56535
Name: retweet, dtype: int64

In [10]:
import hashlib
# Create a hash function to generate anonymized values
def anonymize(value):
    # Convert the value to a string and hash it using SHA256 algorithm
    hashed_value = hashlib.sha256(str(value).encode()).hexdigest()
    # Take the first 8 characters of the hash as the anonymized value
    anonymized_value = hashed_value[:8]
    return anonymized_value

In [11]:
if 'anonymized_user' in working_df.columns:
    working_df.drop('anonymized_user', axis=1, inplace=True)
user_df = pd.DataFrame(working_df['user'].unique(), columns=['user'])
user_df['anonymized_user'] = user_df['user'].apply(anonymize)
working_df = working_df.merge(user_df, on='user', how='left')


In [12]:
working_df.columns

Index(['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count',
       'retweet_count', 'hashtags', 'tweet_df', 'retweet', 'anonymized_user'],
      dtype='object')

In [13]:
working_df.to_csv('tweets2205.csv', index=False)

In [14]:
len(working_df)

118807

In [15]:
working_df['created_date'] = pd.to_datetime(working_df['created_at'], utc=True)
working_df['week_number'] = working_df['created_date'].dt.week
working_df['week_number'].value_counts()

/tmp/ipykernel_660/2145652280.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  working_df['week_number'] = working_df['created_date'].dt.week


20    45385
21    35323
22    33398
23     4700
19        1
Name: week_number, dtype: int64

In [16]:
def get_week_label(week_number):
    if week_number == 20:
        return 'week1'
    elif week_number == 21:
        return 'week2'
    elif week_number == 22:
        return 'week3'
    elif week_number == 23:
        return 'week4'
    else:
        return 'week0'

In [17]:
working_df['week_label'] = working_df['week_number'].apply(get_week_label)

In [18]:
working_df.to_csv('tweeterhashtags/tweets.csv', index=False)

In [19]:
working_df.value_counts('hashtags')

hashtags
#PrimeForLockwoodandCo          15138
#ParamountForLockwoodandCo      10736
#TogetherForLockwoodandCo        9382
#JustRecklessEnough              8720
#BringBackLockwoodandCo          7746
#LockwoodGhostAuditions          7122
#RapiersReady                    6424
#DisneyForLockwoodandCo          5843
#HauntedbyaType3                 5810
#BBCforLockwoodandCo             5391
#WatchPartyatPortlandRow         5253
#FridayNightatPortlandRow        5188
#HauntedWatchParty               4837
#AppleTVforLockwoodandCo         4830
#GhostHuntersWatchParty          4240
#CompleteFictionAppreciation     4172
#CaringforCarlyle                3609
#BunsForBunchurch                3243
#DEPRACisOnTheWay                 612
#DisneySaveLockwood               327
#SkullandCo                       179
#ScullandCo                         5
dtype: int64

In [20]:
try_df = working_df.drop_duplicates()
try_df.value_counts('hashtags')

hashtags
#PrimeForLockwoodandCo          15138
#TogetherForLockwoodandCo        9382
#JustRecklessEnough              8720
#BringBackLockwoodandCo          7746
#ParamountForLockwoodandCo       7451
#DisneyForLockwoodandCo          5812
#HauntedbyaType3                 5807
#BBCforLockwoodandCo             5358
#FridayNightatPortlandRow        5187
#WatchPartyatPortlandRow         5055
#RapiersReady                    4916
#LockwoodGhostAuditions          4867
#HauntedWatchParty               4818
#AppleTVforLockwoodandCo         4722
#GhostHuntersWatchParty          4236
#CompleteFictionAppreciation     4172
#CaringforCarlyle                3554
#BunsForBunchurch                3239
#DEPRACisOnTheWay                 372
#DisneySaveLockwood               327
#SkullandCo                       179
#ScullandCo                         3
dtype: int64